In [12]:
import logging
logging.basicConfig(filename=u"main.log", format=u'%(filename)s[LINE:%(lineno)d]# %(levelname)-8s [%(asctime)s]  %(message)s', level=logging.DEBUG)

logging.debug( u'debug' )
logging.info( u'info' )

import warnings
warnings.filterwarnings('ignore')

In [2]:
# import re
# import time
# from bs4 import BeautifulSoup
# import requests
# import json
# #import Levenshtein as lv
# import numpy as np

# import spacy


# #for findDOI
# import feedparser
# import jellyfish
# import copy
# import os

In [9]:
import pandas as pd
from gensim import corpora, models, similarities

In [2]:
from rss import rss # список рассылки
from findDOI import findDOI # функция поиска doi по названию
from ALT import altmetrics # функция поиска альтметрик (нужно понять, какие альтметрики нам нужны)
from stoplist import stoplist #стоп-лист
from rssParser import Parser # класс, который парсит рсс, в датафрейм, - предлагаю его использовать для наполненния
from findJournal import findJournal # функция поиска названия журнала по названию статьи

In [5]:
# p = Parser(rss)
# df = p.main()
#df.to_csv('example_table.csv')
df = pd.read_csv('example_table.csv')

In [6]:
df.head()

,Unnamed: 0,article_name,doi,journal,summary
0,0,Advances in thermochemical conversion of woody...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
1,1,Fate of degraded pollutants in waste gas biofi...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
2,2,Strain engineering for microbial production of...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
3,3,The state-of-the-art strategies of protein eng...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
4,4,Engineering of filamentous fungi for efficient...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...


In [7]:
# example
findJournal(df.loc[1,]["article_name"])

'Biotechnology Advances'

In [6]:
nlp = spacy.load('en', parser=False, ner=False)  

### Models (gensim)

In [16]:
import gensim 
gensim.__version__

'3.7.3'

In [15]:
new_path = '/home/serge/Desktop/local_dev/testmodels/'

In [16]:
# загружаем словарь, векторное пространство, модель Lsi (вектороное пространство)
# общий корпус натренированный на pubmed 
dictionary = corpora.Dictionary.load('/home/serge/Desktop/local_dev/pubmed/pubmed5.dict')      #Here and later - the first one is PubMed-based LSI-object, other - CUB-based LSI-objects
corpus = corpora.MmCorpus('/home/serge/Desktop/local_dev/pubmed/pubmed5.mm') 
lsi = models.LsiModel.load('/home/serge/Desktop/local_dev/pubmed/pubmed5.lsi')

# химики куб
dictionary_CUB_chem = corpora.Dictionary.load(new_path+"chem_CUB.dict")
lsi_CUB_chem = models.LsiModel.load(new_path+"chem_CUB.lsi")
corpus_CUB_chem = corpora.MmCorpus(new_path+"chem_CUB.mm")

# онко куб
dictionary_CUB_onco = corpora.Dictionary.load(new_path+"cancer_CUB.dict")
lsi_CUB_onco = models.LsiModel.load(new_path+"cancer_CUB.lsi")
corpus_CUB_onco = corpora.MmCorpus(new_path+"cancer_CUB.mm")

# аутоимунные куб
dictionary_CUB_aiz = corpora.Dictionary.load(new_path+"aiz_CUB.dict")
lsi_CUB_aiz = models.LsiModel.load(new_path+"aiz_CUB.lsi")
corpus_CUB_aiz = corpora.MmCorpus(new_path+"aiz_CUB.mm")

# инфекции куб 
dictionary_CUB_inf = corpora.Dictionary.load(new_path+"infect_CUB.dict")
lsi_CUB_inf = models.LsiModel.load(new_path+"infect_CUB.lsi")
corpus_CUB_inf = corpora.MmCorpus(new_path+"infect_CUB.mm")

# офтальмология куб
dictionary_CUB_eye = corpora.Dictionary.load(new_path+"eye_CUB.dict")
lsi_CUB_eye = models.LsiModel.load(new_path+"eye_CUB.lsi")
corpus_CUB_eye = corpora.MmCorpus(new_path+"eye_CUB.mm")

# гететические куб
dictionary_CUB_gene = corpora.Dictionary.load(new_path+"gene_CUB.dict")
lsi_CUB_gene = models.LsiModel.load(new_path+"gene_CUB.lsi")
corpus_CUB_gene = corpora.MmCorpus(new_path+"gene_CUB.mm")

### indexes

In [14]:
index = similarities.MatrixSimilarity(lsi[corpus])          #Indexes
index_CUB_chem = similarities.MatrixSimilarity(lsi_CUB_onco[corpus_CUB_chem])
index_CUB_onco = similarities.MatrixSimilarity(lsi_CUB_onco[corpus_CUB_onco])
index_CUB_aiz = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_aiz])
index_CUB_inf = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_inf])
index_CUB_eye = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_eye])
index_CUB_gene = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_gene])

-----

In [37]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(df.iloc[1,]['summary'], 'html.parser')

In [42]:
soup.select('div p')[0].text

'The fate of the carbon from degraded pollutants in biofiltration is not well understood. The issue of missing carbon needs to be addressed quantitatively to better understand and model biofilter performance. Elucidating the various carbon end-points in various phases should contribute to the fundamental understanding of the degradation kinetics and metabolic pathways as a function of various environmental parameters. This article reviews the implications of key environmental parameters on the carbon end-points. Various studies are evaluated reporting carbon recovery over a multitude of parameters and operational conditions with respect to the analytical measurements and reported distribution of the carbon end-points.'

In [43]:
soup.select('div p')[0].text

'The fate of the carbon from degraded pollutants in biofiltration is not well understood. The issue of missing carbon needs to be addressed quantitatively to better understand and model biofilter performance. Elucidating the various carbon end-points in various phases should contribute to the fundamental understanding of the degradation kinetics and metabolic pathways as a function of various environmental parameters. This article reviews the implications of key environmental parameters on the carbon end-points. Various studies are evaluated reporting carbon recovery over a multitude of parameters and operational conditions with respect to the analytical measurements and reported distribution of the carbon end-points.'

In [ ]:
from enhancment_words import cancerdict, aizdict, infectdict, eyedict, genedict, chemdict
def FirstLevelTags(articlerecord, cancerdict=cancerdict, aizdict=aizdict, infectdict=infectdict, eyedict=eyedict, genedict=genedict, chemdict=chemdict, txt=''):
    
    #articlerecord is a list looking like [journal, title, abstract, doi, altmetrics, topic, chemscore, score]

    cancer_idlist, aiz_idlist, infect_idlist, eye_idlist, gene_idlist, chem_idlist = [], [], [], [], [], []
    cancer_multiplier, aiz_multiplier, infect_multiplier, eye_multiplier, gene_multiplier, chem_multiplier = 0, 0, 0, 0, 0, 0

    txt = articlerecord[2].lower()

    words = corpora.Dictionary([txt.split()])
    ids = words.token2id

    # Checking the presence of the target words from our dictionaries in text and adding them to corresponding lists
    for x in ids.keys():
        for elem in cancerdict:
            if x == elem:
                cancer_idlist.append(ids[x])
        for elem in aizdict:
            if x == elem:
                aiz_idlist.append(ids[x])
        for elem in infectdict:
            if x == elem:
                infect_idlist.append(ids[x])
        for elem in eyedict:
            if x == elem:
                eye_idlist.append(ids[x])
        for elem in genedict:
            if x == elem:
                gene_idlist.append(ids[x])
        for elem in chemdict:
            if x == elem:
                chem_idlist.append(ids[x])
    words = [words.doc2bow(text) for text in [txt.split()]]

    # Calculating the amount of target words in the text
    for e in words[0]:
        for i in cancer_idlist:
            if e[0] == i:
                cancer_multiplier += e[1]
        for i in aiz_idlist:
            if e[0] == i:
                aiz_multiplier += e[1]
        for i in infect_idlist:
            if e[0] == i:
                infect_multiplier += e[1]
        for i in eye_idlist:
            if e[0] == i:
                eye_multiplier += e[1]
        for i in gene_idlist:
            if e[0] == i:
                gene_multiplier += e[1]
        for i in chem_idlist:
            if e[0] == i:
                chem_multiplier += e[1]

    # Constructing a list of amounts of target words for each topic
    multipliers = [cancer_multiplier, aiz_multiplier, infect_multiplier, eye_multiplier, gene_multiplier]

    #Now if we have a long abstact we will try to make LSI magic upon it. Else, we will use keywords approach
    #On this step we try to classify our articles by five medical topics + chemical theme. So, using 2 models: 5-in-1 biological one and chemical one
    if len(txt.split()) >= 50:
        txt = [x for x in txt if x not in stoplist]
        txt = lemmatization(str(txt), allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

        vec_bow = dictionary.doc2bow(txt.split())
        vec_bow_chem = dictionary_CUB_chem.doc2bow(txt.split())

        vec_lsi = lsi[vec_bow]      # convert the query to LSI space
        vec_lsi_chem = lsi_CUB_chem[vec_bow_chem]

        sims = index[vec_lsi]       # perform a similarity query against the corpus
        sims_chem = index[vec_lsi_chem]
        comp_keyword_multiplied = [x*(1+0.08*y) for x, y in zip(sims, multipliers)]   # multiplying the Gensim result to the multiplier from previous steps
        sims = list(enumerate(comp_keyword_multiplied))

        # Tagging the text with one of the first-level-tags basing on the treshold
        number, value = max(sims, key=lambda item: item[1])
        if number == 0 and value >= 0.65:  # Pay attentions to the treshold: it is a matter of choice. It is 0.65 now
            articlerecord.append("cancer")
        elif number == 1 and value >= 0.65:
            articlerecord.append("autoimmunity")
        elif number == 2 and value >= 0.65:
            articlerecord.append("infectious")
        elif number == 3 and value >= 0.65:
            articlerecord.append("ophtalmological")
        elif number == 4 and value >= 0.65:
            articlerecord.append("gene")
        else:
            articlerecord.append("uncategorized")

        #calculating the chemical relation
        score_chem = sum(sims_chem)

    #The keywords approach to title
    else:
        txt = articlerecord[1].lower()
        words = corpora.Dictionary([txt.split()])
        ids = words.token2id

        # Checking the presence of the target words from our dictionaries in text and adding them to corresponding lists
        for x in ids.keys():
            for elem in cancerdict:
                if x == elem:
                    cancer_idlist.append(ids[x])
            for elem in aizdict:
                if x == elem:
                    aiz_idlist.append(ids[x])
            for elem in infectdict:
                if x == elem:
                    infect_idlist.append(ids[x])
            for elem in eyedict:
                if x == elem:
                    eye_idlist.append(ids[x])
            for elem in genedict:
                if x == elem:
                    gene_idlist.append(ids[x])
            for elem in chemdict:
                if x == elem:
                    chem_idlist.append(ids[x])
        words = [words.doc2bow(text) for text in [txt.split()]]

        # Calculating the amount of target words in the text
        for e in words[0]:
            for i in cancer_idlist:
                if e[0] == i:
                    cancer_multiplier += e[1]
            for i in aiz_idlist:
                if e[0] == i:
                    aiz_multiplier += e[1]
            for i in infect_idlist:
                if e[0] == i:
                    infect_multiplier += e[1]
            for i in eye_idlist:
                if e[0] == i:
                    eye_multiplier += e[1]
            for i in gene_idlist:
                if e[0] == i:
                    gene_multiplier += e[1]
            for i in chem_idlist:
                if e[0] == i:
                    chem_multiplier += e[1]

        # Constructing a list of amounts of target words for each topic
        multipliers = [cancer_multiplier, aiz_multiplier, infect_multiplier, eye_multiplier, gene_multiplier, chem_multiplier]
        multipliers = list(enumerate(multipliers))
        number, value = max(multipliers, key=lambda item: item[1])

        if number == 0:
            articlerecord.append("cancer")
        elif number == 1:
            articlerecord.append("autoimmunity")
        elif number == 2:
            articlerecord.append("infectious")
        elif number == 3:
            articlerecord.append("ophtalmological")
        elif number == 4:
            articlerecord.append("gene")
        else:
            articlerecord.append("uncategorized")

        score_chem = chem_multiplier

    articlerecord.append(str(score_chem)+' based on title')
    return articlerecord, txt